[View in Colaboratory](https://colab.research.google.com/github/gowtham91m/cats-and-dogs-classification/blob/master/cats_and_dogs_classificatoin.ipynb)

In [0]:
%%capture
!pip install kaggle

from zipfile import ZipFile
import io, cv2, fnmatch, shutil, os, getpass, subprocess, random

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.optimizers import SGD
from keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

import numpy as np
from time import time
from glob import glob
from sklearn.utils import class_weight

In [0]:
os.chdir('/content')
if 'cats_dogs' in os.listdir('/content'):shutil.rmtree('/content/cats_dogs')
if 'kaggle.txt' not in os.listdir('/content'):
  from google.colab import files
  downloaded = files.upload()

In [19]:
with open('kaggle.txt') as f: key = f.read()
if 'cats_dogs' not in os.listdir('/content'):
  os.mkdir('/content/cats_dogs')
os.chdir('/content/cats_dogs')
os.environ['KAGGLE_USERNAME']="gowham91m"
os.environ['KAGGLE_KEY']=key

!kaggle competitions download -c dogs-vs-cats

  0% 0.00/86.8k [00:00<?, ?B/s]
100% 86.8k/86.8k [00:00<00:00, 36.5MB/s]
 98% 265M/271M [00:04<00:00, 74.9MB/s]
100% 271M/271M [00:04<00:00, 67.5MB/s]
 99% 538M/543M [00:03<00:00, 133MB/s]
100% 543M/543M [00:03<00:00, 176MB/s]


In [20]:
!unzip -q -o train.zip
!unzip -q -o test1.zip

cat_pattern = '*cat.*.jpg'
dog_pattern = '*dog.*.jpg'

images = glob('/content/cats_dogs/train/*.jpg', recursive=True)
cats = fnmatch.filter(images,cat_pattern)
dogs = fnmatch.filter(images,dog_pattern)

os.listdir('/content/cats_dogs')
if 'data' not in os.listdir('/content/cats_dogs'):os.mkdir('/content/cats_dogs/data')
if 'train' not in os.listdir('/content/cats_dogs/data'):os.mkdir('/content/cats_dogs/data/train')
if 'dogs' not in os.listdir('/content/cats_dogs/data/train'):os.mkdir('/content/cats_dogs/data/train/dogs')
if 'cats' not in os.listdir('/content/cats_dogs/data/train'):os.mkdir('/content/cats_dogs/data/train/cats')
  
if 'val' not in os.listdir('/content/cats_dogs/data'):os.mkdir('/content/cats_dogs/data/val')
if 'dogs' not in os.listdir('/content/cats_dogs/data/val'):os.mkdir('/content/cats_dogs/data/val/dogs')
if 'cats' not in os.listdir('/content/cats_dogs/data/val'):os.mkdir('/content/cats_dogs/data/val/cats')

train_dogs_path = '/content/cats_dogs/data/train/dogs'
train_cats_path = '/content/cats_dogs/data/train/cats'

val_dogs_path = '/content/cats_dogs/data/val/dogs'
val_cats_path = '/content/cats_dogs/data/val/cats'

for file in cats: shutil.copy2(file, train_cats_path)
for file in dogs: shutil.copy2(file, train_dogs_path)
  
  
# split train date into train and validation
train_len = len(os.listdir('/content/cats_dogs/data/train/dogs'))
val_len = train_len * 0.3
val_dogs = random.sample(os.listdir(train_dogs_path),int(val_len))
val_cats = random.sample(os.listdir(train_cats_path),int(val_len))


for file in val_dogs:
  try: shutil.move(os.path.join(train_dogs_path,file), val_dogs_path)
  except: pass
for file in val_cats:
  try: shutil.move(os.path.join(train_cats_path,file), val_cats_path)
  except: pass
  
  
  
print(len(os.listdir(train_cats_path)))
print(len(os.listdir(val_cats_path)))

print(len(os.listdir(train_dogs_path)))
print(len(os.listdir(val_dogs_path)))

print('total train samples ', len(os.listdir(train_cats_path)) + len(os.listdir(train_dogs_path)))
print('total train samples ', len(os.listdir(val_cats_path)) + len(os.listdir(val_dogs_path)))

8750
3750
8750
3750
total train samples  17500
total train samples  7500


In [0]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)

batch_size=64
train_generator = train_datagen.flow_from_directory(
        '/content/cats_dogs/data/train',  # this is the target directory
        target_size=(150, 150),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='binary') 

validation_generator = val_datagen.flow_from_directory(
        '/content/cats_dogs/data/val',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')

In [22]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=( 150, 150, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])




start_time = time()
model.fit_generator(
        train_generator,
        #steps_per_epoch=18631 // batch_size,
        epochs=8,
        validation_data=validation_generator,
        #validation_steps=10119 // batch_size
        )
model.save_weights('first_try.h5')

print('time taken ',time()-start_time)

Found 17500 images belonging to 2 classes.
Found 7500 images belonging to 2 classes.
Epoch 1/8
274/274 [==============================] - 177s 647ms/step - loss: 0.6890 - acc: 0.5975 - val_loss: 0.6597 - val_acc: 0.6395
Epoch 2/8
274/274 [==============================] - 171s 623ms/step - loss: 0.5706 - acc: 0.7096 - val_loss: 0.4954 - val_acc: 0.7588
Epoch 3/8
274/274 [==============================] - 171s 623ms/step - loss: 0.5218 - acc: 0.7461 - val_loss: 0.4583 - val_acc: 0.7803
Epoch 4/8
274/274 [==============================] - 171s 623ms/step - loss: 0.4866 - acc: 0.7696 - val_loss: 0.4228 - val_acc: 0.8003
Epoch 5/8
274/274 [==============================] - 171s 625ms/step - loss: 0.4651 - acc: 0.7865 - val_loss: 0.4928 - val_acc: 0.7748
Epoch 6/8
274/274 [==============================] - 172s 629ms/step - loss: 0.4427 - acc: 0.8000 - val_loss: 0.6010 - val_acc: 0.7573
Epoch 7/8
274/274 [==============================] - 172s 627ms/step - loss: 0.4286 - acc: 0.8091 - val_l

In [33]:
from keras.applications.vgg16 import VGG16, preprocess_input

img_rows, img_cols, img_channel = 150, 150, 3
base_model = VGG16(weights='imagenet'
                   ,include_top=False, input_shape=(img_rows, img_cols, img_channel), classes = 2)

add_model = Sequential()
add_model.add(Flatten(input_shape=base_model.output_shape[1:]))
add_model.add(Dense(128, activation='relu'))
add_model.add(Dense(1, activation='sigmoid'))

vgg_model = Model(inputs=base_model.input, outputs=add_model(base_model.output))


for layer in base_model.layers:
    layer.trainable = False
    
    
vgg_model.compile(loss='binary_crossentropy', optimizer=SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])
start_time = time()
vgg_model.fit_generator(
        train_generator,
        epochs=8,
        validation_data=validation_generator,
        #class_weight = class_weights,
        callbacks=[ModelCheckpoint('VGG16-transferlearning.model', monitor='val_acc', save_best_only=True)])
vgg_model.save_weights('vgg16.h5')

print('time taken ',time()-start_time)

Epoch 1/8
274/274 [==============================] - 208s 758ms/step - loss: 0.5588 - acc: 0.7308 - val_loss: 0.4477 - val_acc: 0.8191
Epoch 2/8
274/274 [==============================] - 201s 733ms/step - loss: 0.4333 - acc: 0.8184 - val_loss: 0.3863 - val_acc: 0.8401
Epoch 3/8
274/274 [==============================] - 200s 730ms/step - loss: 0.3848 - acc: 0.8429 - val_loss: 0.3497 - val_acc: 0.8571
Epoch 4/8
274/274 [==============================] - 199s 727ms/step - loss: 0.3606 - acc: 0.8492 - val_loss: 0.3287 - val_acc: 0.8655
Epoch 5/8
274/274 [==============================] - 198s 724ms/step - loss: 0.3408 - acc: 0.8563 - val_loss: 0.3138 - val_acc: 0.8704
Epoch 6/8
274/274 [==============================] - 199s 725ms/step - loss: 0.3288 - acc: 0.8605 - val_loss: 0.3022 - val_acc: 0.8767
Epoch 7/8
274/274 [==============================] - 202s 738ms/step - loss: 0.3171 - acc: 0.8699 - val_loss: 0.2928 - val_acc: 0.8795
Epoch 8/8
274/274 [==============================] - 20